###### Using Service Principal Instead of Storage credential and External Location

In [0]:
# storage_account = "stnyctaxigreen"

# # Fetching from Azure Key Vault
# client_id = dbutils.secrets.get(scope="kv-nyctaxi-scope", key="sp-client-id")
# tenant_id = dbutils.secrets.get(scope="kv-nyctaxi-scope", key="sp-tenant-id")
# client_secret = dbutils.secrets.get(scope="kv-nyctaxi-scope", key="sp-client-secret")

# # Service Principle Authentication
# spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
# spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
# spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
# spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
# spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

##### LOADING FROM LANDING TO SILVER

In [0]:
from pyspark.sql.functions import lit, current_timestamp, col, when
from pyspark.sql.types import TimestampType, IntegerType
from datetime import datetime
from delta.tables import DeltaTable

In [0]:
# Paths and Configuration
volume_path = "/Volumes/nyctaxi/00_landing/data_sources/taxi_zone_lookup/*.csv"
target_table = "nyctaxi.02_silver.taxi_zone_lookup"
storage_path = "abfss://silver@stnyctaxigreen.dfs.core.windows.net/taxi_zone_lookup"

# Read and Transform Source
df_source = spark.read.format("csv").option("header", True).load(volume_path)

df_source = df_source.select(
    col("LocationID").cast(IntegerType()).alias("location_id"),
    col("Borough").alias("borough"),
    col("Zone").alias("zone"),
    col("service_zone"),
    current_timestamp().alias("effective_date"),
    lit(None).cast(TimestampType()).alias("end_date")
)

In [0]:
# Initialize External Table if Missing
if not spark.catalog.tableExists(target_table):
    print(f"Initial run: Creating EXTERNAL table {target_table} at {storage_path}")
    df_source.write.format("delta").option("path", storage_path).saveAsTable(target_table)

###### TEST DATA - DEMO PURPOSE

In [0]:
# Insert new record to the source DataFrame
df_new = spark.createDataFrame([(999, "New Borough", "New Zone", "New Service Zone")], schema="location_id int, borough string, zone string, service_zone string") \
                .withColumn("effective_date", current_timestamp()) \
                .withColumn("end_date", lit(None).cast("timestamp"))

df_source = df_new.union(df_source)


# Updating record for location_id 1
df_source = df_source.withColumn("borough", when(col("location_id")==1, "NEWARK AIRPORT").otherwise(col("borough")))

###### SCD TYPE-2

In [0]:
end_timestamp = datetime.now()
dt = DeltaTable.forName(spark, target_table)

# CHECK 1: Close active rows whose attributes changed
dt.alias("t").merge(
    source = df_source.alias("s"),
    condition = "t.location_id = s.location_id AND t.end_date IS NULL AND (t.borough != s.borough OR t.zone != s.zone OR t.service_zone != s.service_zone)"
).whenMatchedUpdate(
    set = { "t.end_date": lit(end_timestamp).cast(TimestampType()) }
).execute()

# CHECK 2: Insert new updated versions for just-closed IDs
insert_id_list = [row.location_id for row in dt.toDF().filter(f"end_date = '{end_timestamp}'").select("location_id").collect()]

if len(insert_id_list) == 0:
    print("No updated records to insert")
else:   
    dt.alias("t").merge(
        source = df_source.alias("s"),
        condition = f"s.location_id not in ({', '.join(map(str, insert_id_list))})"
    ).whenNotMatchedInsert(
        values = {
            "t.location_id": "s.location_id",
            "t.borough": "s.borough",
            "t.zone": "s.zone",
            "t.service_zone": "s.service_zone",
            "t.effective_date": current_timestamp(),
            "t.end_date": lit(None).cast(TimestampType())
        }
    ).execute()

# CHECK 3: Insert brand-new keys (no historical row in target)
dt.alias("t").merge(
    source = df_source.alias("s"),
    condition = "t.location_id = s.location_id"
).whenNotMatchedInsert(
    values = {
        "t.location_id": "s.location_id",
        "t.borough": "s.borough",
        "t.zone": "s.zone",
        "t.service_zone": "s.service_zone",
        "t.effective_date": current_timestamp(),
        "t.end_date": lit(None).cast(TimestampType())
    }
).execute()

print(f"Total records in {target_table}: {spark.read.table(target_table).count()}")